In [15]:
# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


#**Web Scraping** 
- RCurl
- httr
- XML
- rvest

## Web Crawling - "rastejar" por web page ou site buscando dados


## Web Scraping - "raspar" os dados de interesse  de uma web page

In [0]:
%%R
# Pacote rvest - útil para quem não conhece HTML e CSS
install.packages('rvest')
library(rvest) 

install.packages("data.table")
library(data.table)

library(stringr)
library(dplyr)
library(lubridate) #pacote para tratar de datas
library(readr) #pacote para fazer a leitura de arquivos

In [17]:
%%R
# Leitura da web page - Retorna um documento xml para atribuir a um objeto em R
webpage <- read_html("https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html")
webpage #documento xml do tipo "semi estruturado"

{html_document}
<html lang="en" class="no-js page-interactive section-opinion page-theme-standard tone-opinion page-interactive-default limit-small layout-xlarge app-interactive" itemid="https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html" itemtype="http://schema.org/NewsArticle" itemscope="" xmlns:og="http://opengraphprotocol.org/schema/">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body>\n<style>\n.lt-ie10 .messenger.suggestions {\n  display: block !imp ...


#**Extraindo os arquivos html com html_nodes()** - {rvest}
## Cada elemento n web tem o seguinte formato em html:

In [18]:
%%R
# <span class="short-desc"><strong> DATE </strong> LIE <span class="short-truth"><a href="URL"> EXPLANATION </a></span></span>
# ?html_nodes
results <- webpage %>% html_nodes(".short-desc") #extrair tudo que possuir classe ".short-desc" através da função html_nodes()
results 


{xml_nodeset (180)}
 [1] <span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Ira ...
 [2] <span class="short-desc"><strong>Jan. 21 </strong>“A reporter for Time m ...
 [3] <span class="short-desc"><strong>Jan. 23 </strong>“Between 3 million and ...
 [4] <span class="short-desc"><strong>Jan. 25 </strong>“Now, the audience was ...
 [5] <span class="short-desc"><strong>Jan. 25 </strong>“Take a look at the Pe ...
 [6] <span class="short-desc"><strong>Jan. 25 </strong>“You had millions of p ...
 [7] <span class="short-desc"><strong>Jan. 25 </strong>“So, look, when Presid ...
 [8] <span class="short-desc"><strong>Jan. 26 </strong>“We've taken in tens o ...
 [9] <span class="short-desc"><strong>Jan. 26 </strong>“I cut off hundreds of ...
[10] <span class="short-desc"><strong>Jan. 28 </strong>“The coverage about me ...
[11] <span class="short-desc"><strong>Jan. 29 </strong>“The Cuban-Americans,  ...
[12] <span class="short-desc"><strong>Jan. 30 </strong>“Only 109 people out o 

## Acima temos uma lista com a classe da tag .short-desc especificada em html_nodes



---



---



### Abaixo estamos pegando o comprimento total do conjunto de dados junto com uma lista, criando um vetor com esta seleção e e atribuindo a "records"

In [19]:
%%R
# Construindo o dataset
records <- vector("list", length = length(results)) #pegando comprimento total do conjunto
records
# esta lista NULL receberá todas as alterações que iremos processar a partir de agora

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

[[75]]
NULL

[[76]]
NULL

[[77]]
NULL

[[78]]
N

## loop for contém toda a inteligência que começa em i e percorre todo o comprimento de results [180]

In [0]:
%%R
for (i in seq_along(results)) {
  date <- str_c(results[i] %>% #extrair a data
                  html_nodes("strong") %>% #a partir do momento que encontra html strong
                  html_text(trim = TRUE), ', 2017') #extrair tudo que há na TAG e remove o espaço com trim
  
  lie <- str_sub(xml_contents(results[i])[2] %>% html_text(trim = TRUE), 2, -2)#extrair a mentira
  #extrair uma string com o str_sub() e converter results em xml_contents para facilitar o processamento
  #feito isso usar html_text para buscar o texto
  
  explanation <- str_sub(results[i] %>% #extrair a explicação
                           html_nodes(".short-truth") %>% #short-truth mostra exatamente a explicação dentro
                           html_text(trim = TRUE), 2, -2) #assim que acha tudo que há dentro de short-truth, retornará o texto

  
  url <- results[i] %>% html_nodes("a") %>% html_attr("href")#extrair tudo que for link html, ou seja, "href"
  #href aponta para o link 
  
  records[[i]] <- data_frame(date = date, lie = lie, explanation = explanation, url = url)
} #na sequência gravar um data_frame com tudo que foi extraído no objeto records



In [0]:
%%R
# Dataset final
df <- bind_rows(records) #bind_rows para juntar todas as linhas do objeto records


In [0]:
%%R
# Transformando o campo data em "caracter" para o formato Date em R
df$date <- mdy(df$date) # ajustar a data, visto que ao extrair a data converteu para caracter

In [0]:
%%R
# Exportando para CSV
write_csv(df, "mentiras_trump.csv")# salvar o resultado em um arquivo csv

In [32]:
%%R
# Lendo os dados
df <- read_csv("mentiras_trump.csv")# agora ler o arquivo csv 
df #imprimir dataframe


R[write to console]: Parsed with column specification:
cols(
  date = col_date(format = ""),
  lie = col_character(),
  explanation = col_character(),
  url = col_character()
)



# A tibble: 180 x 4
   date       lie                  explanation               url                
   <date>     <chr>                <chr>                     <chr>              
 1 2017-01-21 I wasn't a fan of I… He was for an invasion b… https://www.buzzfe…
 2 2017-01-21 A reporter for Time… Trump was on the cover 1… http://nation.time…
 3 2017-01-23 Between 3 million a… There's no evidence of i… https://www.nytime…
 4 2017-01-25 Now, the audience w… Official aerial photos s… https://www.nytime…
 5 2017-01-25 Take a look at the … The report never mention… https://www.nytime…
 6 2017-01-25 You had millions of… The real number is less … https://www.nytime…
 7 2017-01-25 So, look, when Pres… There were no gun homici… https://www.dnainf…
 8 2017-01-26 We've taken in tens… Vetting lasts up to two … https://www.nytime…
 9 2017-01-26 I cut off hundreds … Most of the cuts were al… https://www.washin…
10 2017-01-28 The coverage about … It never apologized.      https://www.nytime…
# … with